In [ ]:
import numpy as np
from pygsti.baseobjs import Basis

np.set_printoptions(precision=1, linewidth=1000)

In [ ]:
# 1-qubit base case
pp = Basis.cast('PP', dim=4)
pauliNames = ["I", "X", "Y", "Z"]
## HEY THIS NEEDS TO BE PER QUBIT +/-!!!
initialStates = ["X+", "X-", "Y+","Y-", "Z+", "Z-"]


In [ ]:
# Commutator Helper Functions
def commute(mat1, mat2):
    return mat1@mat2 + mat2@mat1

def anti_commute(mat1, mat2):
    return mat1@mat2 - mat2@mat1

In [ ]:
# Hamiltonian Error Generator
def hamiltonian_error_generator(initial_state, pauli_index, pauliDict, numQubits):
    return -1j*pauliDict[pauli_index]@initial_state@pauliDict["I"*numQubits] + 1j*pauliDict["I"*numQubits]@initial_state@pauliDict[pauli_index]

# Stochastic Error Generator
def stochastic_error_generator(initial_state, pauli_index, pauliDict):
    return pauliDict[pauli_index]@initial_state@pauliDict[pauli_index] - pauliDict["I"]@initial_state@pauliDict["I"]

# Pauli-correlation Error Generator
def pauli_correlation_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] + pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] - 0.5 * commute(commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state)

# Anti-symmetric Error Generator
def anti_symmetric_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return 1j*(pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] - pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] + 0.5 * commute(anti_commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state))

In [ ]:
# Convert basis; 1-q only at the moment
def convert_to_pauli(matrix, numQubits):
    pauliNames1Q = ["I", "X", "Y", "Z"]
    # Hard force to 1- or 2-qubit
    if numQubits == 1:
        pauliNames = pauliNames1Q
    elif numQubits == 2:
        pauliNames = [''.join(name) for name in product(pauliNames1Q, pauliNames1Q)]
    translationMatrix = pp.from_std_transform_matrix
    coefs = np.real_if_close(np.dot(translationMatrix,matrix.flatten()))
    return [(a,b) for (a,b) in zip(coefs,pauliNames) if abs(a) > 0.0001]

In [ ]:
# Compute the set of measurable effects of Hamiltonian error generators operating on two qubits in each of the specified eigenstates
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()
numQubits = 1
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "+":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) + 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)
        elif state[-1] == "-":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) - 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)

# Convert measurable effects into coefficients
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(hamiltonianErrorOutputs[key], numQubits)

for key in hamiltonianErrorOutputs:
    print(key, "\n", hamiltonianErrorOutputs[key])

In [ ]:
# Compute the set of measurable effects of stochastic error generators operating on a single qubit in each of the specified eigenstates
stochasticIndices = ["X", "Y", "Z"]
stochasticErrorOutputs = dict()
for index in stochasticIndices:
    for state in initialStates:
        if state[-1] == "+":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp) + 0.5*stochastic_error_generator(pp[state[0]], index, pp)
        elif state[-1] == "-":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp) - 0.5*stochastic_error_generator(pp[state[0]], index, pp)

# Convert measurable effects into coefficients
for key in stochasticErrorOutputs:
    stochasticErrorOutputs[key] = convert_to_pauli(stochasticErrorOutputs[key], numQubits)
for key in stochasticErrorOutputs:
    print(key, "\n", stochasticErrorOutputs[key])


In [ ]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
pauliCorrelationIndices = ["XY", "XZ", "YZ"]
pauliCorrelationErrorOutputs = dict()
for index in pauliCorrelationIndices:
    for state in initialStates:
        if state[-1] == "+":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) + 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) - 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in pauliCorrelationErrorOutputs:
    pauliCorrelationErrorOutputs[key] = convert_to_pauli(pauliCorrelationErrorOutputs[key], numQubits)
for key in pauliCorrelationErrorOutputs:
    print(key, "\n", pauliCorrelationErrorOutputs[key])


In [ ]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
antiSymmetricIndices = ["XY", "XZ", "YZ"]
antiSymmetricErrorOutputs = dict()
for index in antiSymmetricIndices:
    for state in initialStates:
        if state[-1] == "+":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) + 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) - 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in antiSymmetricErrorOutputs:
    antiSymmetricErrorOutputs[key] = convert_to_pauli(antiSymmetricErrorOutputs[key], numQubits)
for key in antiSymmetricErrorOutputs:
    print(key, "\n", antiSymmetricErrorOutputs[key])


In [ ]:
# 2-Qubit case
from itertools import product
pp = Basis.cast('PP', dim=16)
pauliNames1Q = ["I", "X", "Y", "Z"]
pauliNames = [''.join(name) for name in product(pauliNames1Q, pauliNames1Q)]
initialStates = [''.join((name, '+')) for name in pauliNames[1:]] + [''.join((name, '-')) for name in pauliNames[1:]]

In [ ]:
print(pauliNames)
print(initialStates)

In [ ]:
# Compute the set of measurable effects of Hamiltonian error generators operating on two qubits in each of the specified eigenstates
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()
numQubits = 2
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "+":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) + 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)
        elif state[-1] == "-":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) - 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)

# Convert measurable effects into coefficients
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(hamiltonianErrorOutputs[key], numQubits)

for key in hamiltonianErrorOutputs:
    print(key, "\n", hamiltonianErrorOutputs[key])

In [58]:
import pygsti
from pygsti.extras import idletomography as idt

n_qubits = 1
gates = ["Gi","Gx","Gy","Gcnot"]
max_lengths = [1,2,4,8]
pspec = pygsti.processors.QubitProcessorSpec(n_qubits, gates, geometry='line', nonstd_gate_unitaries={():1})

mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(n_qubits, max_lengths, paulidicts, maxweight=1)
print(len(idle_experiments), "idle tomography experiments for %d qubits" % n_qubits)

36 idle tomography experiments for 1 qubits


In [44]:
mdl_target.operation_blks['layers'].keys()

odict_keys([Label(('Gi', 0)), Label(('Gx', 0)), Label(('Gy', 0))])

In [45]:
mdl_target.probabilities(pygsti.circuits.Circuit([[]],line_labels=(0,)))

OutcomeLabelDict([(('0',), 1.0), (('1',), 1.0146536357569526e-17)])

In [47]:
print(mdl_target)

layers:rho0 = Computational Z-basis state vec for 1 qubits w/z-values: [0]

layers:Mdefault = Computational(Z)-basis POVM on 1 qubits and filter None


gates:Gi = 
StaticStandardOp with name Gi and evotype densitymx_slow


gates:Gx = 
StaticStandardOp with name Gx and evotype densitymx_slow


gates:Gy = 
StaticStandardOp with name Gy and evotype densitymx_slow


gates:Gcnot = 
StaticStandardOp with name Gcnot and evotype densitymx_slow


layers:Gi:0 = 
StaticStandardOp with name Gi and evotype densitymx_slow


layers:Gx:0 = 
StaticStandardOp with name Gx and evotype densitymx_slow


layers:Gy:0 = 
StaticStandardOp with name Gy and evotype densitymx_slow







In [72]:
#Generate some data (takes ~5min w/10Q)
from pygsti.baseobjs import Label 
mdl_datagen = pygsti.models.create_crosstalk_free_model(pspec,
                                                        lindblad_error_coeffs={'Gi': {'HX': 0.01, 'SX': 0.01}})
ds = pygsti.data.simulate_data(mdl_datagen, updated_ckt_list, 100000, seed=8675309)

c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\circuits\circuit.py:816: UserWarning: Editable circuit is being converted to read-only mode in order to hash it.  You should call circuit.done_editing() beforehand.
  _warnings.warn(("Editable circuit is being converted to read-only"


In [60]:
print(idle_experiments)

[Circuit(Gy:0[]Gy:0@(0)), Circuit(Gy:0[][]Gy:0@(0)), Circuit(Gy:0[][][][]Gy:0@(0)), Circuit(Gy:0[][][][][][][][]Gy:0@(0)), Circuit(Gx:0[]Gx:0@(0)), Circuit(Gx:0[][]Gx:0@(0)), Circuit(Gx:0[][][][]Gx:0@(0)), Circuit(Gx:0[][][][][][][][]Gx:0@(0)), Circuit([]@(0)), Circuit([][]@(0)), Circuit([][][][]@(0)), Circuit([][][][][][][][]@(0)), Circuit(Gy:0Gy:0Gy:0[]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][][][]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][][][][][][][]Gy:0Gy:0Gy:0@(0)), Circuit(Gx:0Gx:0Gx:0[]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][][][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][][][][][][][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][][][]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][][][][][][][]Gx:0Gx:0@(0)), Circuit([]Gx:0@(0)), Circuit([][]Gx:0@(0)), Circuit([][][][]Gx:0@(0)), Circuit([][][][][][][][]Gx:0@(0)), Circuit(Gy:0[]@(0)), Circuit(Gy:0

In [63]:
idle_experiments[0][1]

Label(())

In [71]:
updated_ckt_list= []
for ckt in idle_experiments:
    new_ckt= ckt.copy(editable=True)
    for i,lbl in enumerate(ckt):
        if lbl == Label(()):
            new_ckt[i] = Label(('Gi',0))
    updated_ckt_list.append(new_ckt)

In [73]:
print(ds)

Gy:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0Gi:0Gx:0@(0)  :  {('0',): 524.0, ('1',): 99476.0}
Gx:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1001.0, ('1',): 98999.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1977.0, ('1',): 98023.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 4124.0, ('1',): 95876.0}
Gi:0@(0)  :  {('0',): 99501.0, ('1',): 499.0}
Gi:0Gi:0@(0)  :  {('0',): 98939.0, ('1',): 1061.0}
Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 98007.0, ('1',): 1993.0}
Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 95854.0, ('1',): 4146.0}
Gy:0Gy:0Gy:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:

In [77]:
#Run idle tomography (takes ~1.5min w/10Q)
results = idt.do_idle_tomography(n_qubits, ds, max_lengths, paulidicts, maxweight=1, advanced_options={"jacobian mode":"together"}, idle_string='Gi:0')

In [23]:
print(results.observed_rate_infos.keys())

dict_keys(['samebasis', 'diffbasis'])


In [24]:
print(len(results.observed_rate_infos['samebasis']))

6


In [25]:
print(len(results.pauli_fidpairs['samebasis']))

6


In [35]:
results.pauli_fidpairs['samebasis']

[(State[+X], State[-X]),
 (State[-Y], State[+Y]),
 (State[+Z], State[+Z]),
 (State[-X], State[+X]),
 (State[+Y], State[-Y]),
 (State[-Z], State[-Z])]

In [36]:
results.pauli_fidpairs['diffbasis']

[(State[+Z], State[+Y]), (State[+X], State[+Z]), (State[-Y], State[-X])]

In [78]:
results.observed_rate_infos['diffbasis']

[OrderedDict([(NQPauliOp[ Y],
               {'rate': -0.01249839683278231,
                'fit_order': 1,
                'fitCoeffs': array([-0.0124984 , -0.00057968]),
                'data': [-0.00998, -0.02726, -0.05346, -0.0991],
                'errbars': [0.003162120174186933,
                 0.003161102485526213,
                 0.003157755576988187,
                 0.003146711283228889],
                'weights': [632.4869664443099,
                 632.6904723679822,
                 633.3608680201303,
                 635.5834462698276],
                'jacobian row': array([-1.,  0.,  0.]),
                'affine jacobian row': array([0, 2, 0])})]),
 OrderedDict([(NQPauliOp[ Z],
               {'rate': 9.791318297408492e-05,
                'fit_order': 1,
                'fitCoeffs': array([ 9.7913183e-05, -2.7321772e-03]),
                'data': [-0.00298, -0.00312, -0.00086, -0.0025],
                'errbars': [0.0031622636189919395,
                 0.00316226

In [79]:
results.observed_rate_infos['samebasis']

[OrderedDict([(NQOutcome[0],
               {'rate': 0.0,
                'fit_order': 1,
                'fitCoeffs': array([0., 0.]),
                'data': [0.0, 0.0, 0.0, 0.0],
                'errbars': [0.0, 0.0, 0.0, 0.0],
                'weights': [100001.49999375004,
                 100001.49999375004,
                 100001.49999375004,
                 100001.49999375004],
                'jacobian row': array([ 0,  1,  1, -1,  0,  0])})]),
 OrderedDict([(NQOutcome[0],
               {'rate': 0.005082511835631539,
                'fit_order': 1,
                'fitCoeffs': array([5.08251184e-03, 8.29046294e-07]),
                'data': [0.00524, 0.01001, 0.01977, 0.04124],
                'errbars': [0.00022830992970083452,
                 0.0003147983465649081,
                 0.0004402175269114123,
                 0.0006288025318015188],
                'weights': [4375.903079386673,
                 3175.0986822727577,
                 2271.063784755441,
        

In [37]:
results.error_list

[NQPauliOp[ X], NQPauliOp[ Y], NQPauliOp[ Z]]

In [39]:
print(results)

Idle Tomography Results
Intrinsic stochastic rates: 
   X: 5.84863e-20
   Y: -3.35252e-20
   Z: -2.49611e-20
Intrinsic affine rates: 
   X: 5.79414e-21
   Y: 5.21083e-21
   Z: -1.09352e-19
Intrinsic hamiltonian rates:
   X: 0.000410094
   Y: -0.000521394
   Z: -0.000456001



In [41]:
print(ds)

Gy:0[]Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0[][]Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0[][][][]Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0[][][][][][][][]Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0[]Gx:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0[][]Gx:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0[][][][]Gx:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0[][][][][][][][]Gx:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
[]@(0)  :  {('0',): 100000.0, ('1',): 0.0}
[][]@(0)  :  {('0',): 100000.0, ('1',): 0.0}
[][][][]@(0)  :  {('0',): 100000.0, ('1',): 0.0}
[][][][][][][][]@(0)  :  {('0',): 100000.0, ('1',): 0.0}
Gy:0Gy:0Gy:0[]Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0[][]Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0[][][][]Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0[][][][][][][][]Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0Gx:0Gx:0[]Gx:0Gx:0Gx:0@(0)  :  {('0',): 0.0, ('1',): 100000.